In [1]:
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import plotly.graph_objects as go
import sys
sys.path.append('../../assets/python/')
import tfb

METADATA = {'Contributor': 'T. Dunn'}
SAVEFIGS = False

df0 = pd.read_csv('../../assets/data/monsters.csv')
df0 = df0.astype({'Book': 'category', 'Category': 'category', 'Legendary': 'category', 'Type': 'category', 'Size': 'category'})
df0['Str PB'] = df0['Str Prof']*df0['PB']
df0['Dex PB'] = df0['Dex Prof']*df0['PB']
df0['Con PB'] = df0['Con Prof']*df0['PB']
df0['Wis PB'] = df0['Wis Prof']*df0['PB']
df0['Int PB'] = df0['Int Prof']*df0['PB']
df0['Cha PB'] = df0['Cha Prof']*df0['PB']
print('Includes {} monsters in database'.format(len(df0.index.to_list())))

Includes 2144 monsters in database


In [2]:
def plot_mean_fit(fig, df0, xCol, yCol, name, line_color):
    # plot mean
    dfM = df0[[xCol, yCol]].groupby(xCol).mean().reset_index()
    fig.add_trace(go.Scatter(
        x=dfM[xCol], y=dfM[yCol],
        mode='markers', 
        name=f'{name}',
        legendgroup=name, 
        line_color=line_color,
        hovertemplate='<b>' + name + '</b><br>' + 'CR %{x:,.0f}<br>' + 'Mean %{y:,.1f}<br>' + '<extra></extra>'
    ))
    coefs = np.polyfit(dfM[xCol], dfM[yCol], 1)
    poly = np.poly1d(coefs)
    fig.add_trace(go.Scatter(
        x=dfM[xCol], y=poly(dfM[xCol]),
        mode='lines', 
        name=f'{name}',
        legendgroup=name, 
        line_color=line_color,
        hoverinfo='skip',
        showlegend=False,
    ))

In [3]:
def plot_data_and_fit(fig, x, y, **kwargs):
    print_coefficients = kwargs.pop('print_coefficients', False)
    # plot data
    fig.add_trace(go.Scatter(
        x=x, y=y,
        mode='markers', 
        **kwargs,
    ))

    # plot fit
    kwargs['hoverinfo'] = 'skip'
    kwargs['showlegend'] = False
    if kwargs.get('line', None):
        kwargs['line']['dash'] = 'solid'

    coefs = np.polyfit(x, y, 1)
    poly = np.poly1d(coefs)
    fig.add_trace(go.Scatter(
        x=x, y=poly(x),
        mode='lines', 
        **kwargs,
    ))
    if print_coefficients: 
        print(f'CR >= 1: {coefs[1]:.2f} + {coefs[0]:.2f}*CR')
    return fig

In [4]:
# Fig. 1: Saving throw modifier
columns = ['Str Save','Dex Save','Con Save','Int Save','Wis Save','Cha Save']
colors = ['#d62728','#9467bd','#8c564b','#1f77b4','#2ca02c','#ff7f0e']

df1 = df0[df0['Category'].isin(['generic','legendary','unique']) & df0['CR'].between(1, 30)]
dfM = df1[['CR'] + columns].groupby(['CR']).mean().reset_index()

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=70, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='CR',
            range=[0,31],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='saving throw bonus',
            range=[-2,19],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

for column, color in zip(columns, colors):
    #plot_mean_fit(fig, df1, 'CR', column, column, color)
    plot_data_and_fit(
        fig, 
        dfM['CR'], dfM[column],
        print_coefficients=True,
        name=column,
        line=dict(color=color),
        hovertemplate=
            f'<b>{column}</b><br>'+
            'CR %{x}<br>'+
            'mean %{y:,.1f}' + 
            '<extra></extra>'
    )

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name='fig-saving-throw-modifier-vs-cr-large')
    tfb.save_fig_html(fig, format='small', name='fig-saving-throw-modifier-vs-cr-small')

CR >= 1: 1.01 + 0.32*CR
CR >= 1: 0.47 + 0.26*CR
CR >= 1: 0.45 + 0.50*CR
CR >= 1: -1.53 + 0.35*CR
CR >= 1: -0.44 + 0.47*CR
CR >= 1: -1.84 + 0.51*CR


In [5]:
# Fig. 2: Ability score modifier
columns = ['Str Mod','Dex Mod','Con Mod','Int Mod','Wis Mod','Cha Mod']
colors = ['#d62728','#9467bd','#8c564b','#1f77b4','#2ca02c','#ff7f0e']

df1 = df0[df0['Category'].isin(['generic','legendary','unique']) & df0['CR'].between(1, 30)]
dfM = df1[['CR'] + columns].groupby(['CR']).mean().reset_index()

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=70, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='CR',
            range=[0,31],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='ability score modifier',
            range=[-2,11],
            tick0=0, dtick=2,
            minor=dict(tick0=0, dtick=1),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

for column, color in zip(columns, colors):
    #plot_mean_fit(fig, df1, 'CR', column, column, color)
    plot_data_and_fit(
        fig, 
        dfM['CR'], dfM[column],
        print_coefficients=True,
        name=column,
        line=dict(color=color),
        hovertemplate=
            f'<b>{column}</b><br>'+
            'CR %{x}<br>'+
            'mean %{y:,.1f}' + 
            '<extra></extra>'
    )

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name='fig-ability-score-modifier-vs-cr-large')
    tfb.save_fig_html(fig, format='small', name='fig-ability-score-modifier-vs-cr-small')

CR >= 1: 0.81 + 0.29*CR
CR >= 1: 1.27 + 0.04*CR
CR >= 1: 0.96 + 0.27*CR
CR >= 1: -1.27 + 0.23*CR
CR >= 1: -0.04 + 0.19*CR
CR >= 1: -1.15 + 0.28*CR


In [7]:
# Fig. 3: Saving throw proficiency
columns = ['Str Prof','Dex Prof','Con Prof','Int Prof','Wis Prof','Cha Prof']
colors = ['#d62728','#9467bd','#8c564b','#1f77b4','#2ca02c','#ff7f0e']

df1 = df0[df0['Category'].isin(['generic','legendary','unique']) & df0['CR'].between(1, 30)]

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=70, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='CR',
            range=[0,31],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='saving throw proficiency probability',
            range=[-0.01,1.01],
            tick0=0, dtick=0.2,
            minor=dict(tick0=0, dtick=0.1),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

for column, color in zip(columns, colors):
    plot_mean_fit(fig, df1, 'CR', column, column, color)

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name='fig-saving-throw-proficiency-vs-cr-large')
    tfb.save_fig_html(fig, format='small', name='fig-saving-throw-proficiency-vs-cr-small')

In [8]:
# Fig. 4: Saving throw proficiency bonus
columns = ['Str PB','Dex PB','Con PB','Int PB','Wis PB','Cha PB']
colors = ['#d62728','#9467bd','#8c564b','#1f77b4','#2ca02c','#ff7f0e']

df1 = df0[df0['Category'].isin(['generic','legendary','unique']) & df0['CR'].between(1, 30)]

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=70, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='CR',
            range=[0,31],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='saving throw bonus',
            range=[-2,11],
            tick0=0, dtick=2,
            minor=dict(tick0=0, dtick=1),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

for column, color in zip(columns, colors):
    plot_mean_fit(fig, df1, 'CR', column, column, color)

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name='fig-saving-throw-proficiency-bonus-vs-cr-large')
    tfb.save_fig_html(fig, format='small', name='fig-saving-throw-proficiency-bonus-vs-cr-small')